**This project uses the transformers library for sentiment analysis and summarization. It calculates the sentiment of each review, computes an average sentiment score, generates a summary of all reviews, and provides a final verdict based on the average score. Finally, it prints the summary and the verdict.**



**This code block uses requests and BeautifulSoup to scrape up to 10 customer reviews from an Amazon product URL provided by the user, then prints the collected reviews.**


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_amazon_reviews(product_url, max_reviews=10):
    reviews = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9"
    }

    page = 1
    while len(reviews) < max_reviews and page <= 5:
        paged_url = f"{product_url}?pageNumber={page}"
        try:
            resp = requests.get(paged_url, headers=headers, timeout=10)
            if resp.status_code != 200:
                print(f"Failed to fetch page {page}")
                break
        except requests.exceptions.Timeout:
            print("Request timed out.")
            break

        soup = BeautifulSoup(resp.text, 'lxml')
        review_divs = soup.find_all('span', {'data-hook': 'review-body'})

        if not review_divs:
            print("No reviews found or no more reviews.")
            break

        for review in review_divs:
            text = review.get_text(strip=True).replace("Read more", "").strip()
            if text and text not in reviews:
                reviews.append(text)
                if len(reviews) >= max_reviews:
                    break

        page += 1

    return reviews

product_url = input("Enter Amazon product reviews URL: ").strip()
reviews = get_amazon_reviews(product_url, max_reviews=10)

for i, r in enumerate(reviews, 1):
    print(f"\nReview {i}:\n{r}")


Enter Amazon product reviews URL: https://www.amazon.in/SUGAR-Cosmetics-Matte-Crayon-Lipstick/dp/B09W5GJNJN/ref=pd_ci_mcx_mh_mcx_views_0_title?pd_rd_w=tdqOO&content-id=amzn1.sym.529d03fa-575b-4f2b-a4d6-0c02eabf0a7e%3Aamzn1.symc.45dc5f4c-d617-4dba-aa26-2cadef3da899&pf_rd_p=529d03fa-575b-4f2b-a4d6-0c02eabf0a7e&pf_rd_r=JQ1DHK3HPKY9TFMV075C&pd_rd_wg=okwlO&pd_rd_r=15acd3a0-d11a-402c-bedd-a9bd29d657f7&pd_rd_i=B09W5GJNJN&th=1

Review 1:
I’m obsessed with SUGAR’s Matte As Hell Crayon Lipstick in 15 Stephanie Plum (Plum Mauve)! This bold mauve-plum shade is my go-to for everything from work to nights out—it’s vibrant yet classy and flatters my medium Indian skin tone perfectly. The 2.8g crayon applies like a dream, giving full-coverage color in one swipe that lasts 8+ hours without fading. Its creamy, hydrating formula keeps my lips soft, and the silky matte finish feels luxe. The included sharpener ensures precise application, and the smudge-proof wear survives coffee and chats! Packaged in a 

In [ ]:
!pip install transformers torch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.3 MB/s eta 0:00:00


**Performing sentiment analysis on collected reviews. Loading a pre-trained model and tokenizer. Predicting the sentiment (negative, neutral, positive) for each review.**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load pretrained model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ['negative', 'neutral', 'positive']

def analyze_sentiment(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    # Predict
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = F.softmax(logits, dim=1)
    sentiment = labels[probs.argmax()]
    return sentiment

# Example: Sentiment of all reviews
for i, review in enumerate(reviews, 1):
    sentiment = analyze_sentiment(review)
    print(f"\nReview {i}:")
    print(review)
    print(f"Sentiment: {sentiment}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]


Review 1:
I’m obsessed with SUGAR’s Matte As Hell Crayon Lipstick in 15 Stephanie Plum (Plum Mauve)! This bold mauve-plum shade is my go-to for everything from work to nights out—it’s vibrant yet classy and flatters my medium Indian skin tone perfectly. The 2.8g crayon applies like a dream, giving full-coverage color in one swipe that lasts 8+ hours without fading. Its creamy, hydrating formula keeps my lips soft, and the silky matte finish feels luxe. The included sharpener ensures precise application, and the smudge-proof wear survives coffee and chats! Packaged in a chic black case, it’s a steal at Rs. 849 for such stunning quality!
Sentiment: positive

Review 2:
👎👎👎Really disappointed with this purchase from Sugar. The product arrived damaged, and there’s no option for return or replacement, which is incredibly frustrating. I expected better quality control and customer support from such a well-known brand. It’s unacceptable to pay full price and receive a broken item with no way 

**This code is analyzing the sentiment of collected Amazon reviews, calculating an overall sentiment score, generating a summary of the reviews, and providing a final buy/don't buy verdict based on the sentiment analysis.**

In [ ]:
from transformers import pipeline
from statistics import mean


# STEP 2: Perform sentiment analysis on each review
sentiment_pipeline = pipeline("sentiment-analysis")

sentiments = sentiment_pipeline(reviews)

# STEP 3: Convert sentiment labels to numeric scores
def sentiment_score(label):
    return {"POSITIVE": 1, "NEGATIVE": -1, "NEUTRAL": 0}.get(label, 0)

scores = [sentiment_score(result["label"]) for result in sentiments]
average_score = mean(scores)

# STEP 4: Generate a summary of all reviews
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
all_text = " ".join(reviews)
summary_output = summarizer(all_text, max_length=60, min_length=20, do_sample=False)[0]["summary_text"]

# STEP 5: Make a final verdict based on average score
if average_score > 0.5:
    verdict = "✅ Buy — Most reviews are positive."
elif average_score >= 0:
    verdict = "⚠️ Mixed — Consider checking more details before buying."
else:
    verdict = "❌ Not Recommended — Mostly negative reviews."

# STEP 6: Output everything
print("📝 Summary of Reviews:")
print(summary_output)

print("\n📢 Final Verdict:")
print(verdict)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
Device set to use cuda:0


📝 Summary of Reviews:
Sugar’s Matte As Hell Crayon Lipstick in 15 Stephanie Plum (Plum Mauve)! This bold mauve-plum shade is my go-to for everything from work to nights out. The 2.8g crayon applies like a dream

📢 Final Verdict:
⚠️ Mixed — Consider checking more details before buying.
